# Introduction to matplotlib

`matplotlib` is the Python plotting package to rule them all. Not because it's the best. Or the easiest to use. Or the fastest. Or... wait, why is it the number 1 plotting package? Nobody knows! But it's everywhere, and making basic plots is... fine. It's really fine.

This is the `pyplot` interface to `matplotlib`. It's the easiest one to use, but it's less flexible than the so-called 'object-oriented' interface.

Here's the official tutorial for the `pyplot` interface: https://matplotlib.org/tutorials/introductory/pyplot.html

Producing the same plot with the object-oriented interface looks like this:

For the rest of this lesson, we're going to use the object-oriented approach.

## Load some data

In [ ]:
import pandas as pd

url = "https://github.com/scienxlab/datasets/raw/refs/heads/main/kgs/panoma_data.xlsx"

df = pd.read_excel(url, sheet_name='data')
df.head()

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">

<h3>EXERCISE</h3>

- Make the following variables, using only the SHRIMPLIN well. For each one, add `.to_numpy()` at the end so we end up with a NumPy array, not a Pandas object.
  - `gr`: the GR log.
  - `phi`: the PHIND log.
  - `depth`: the depth column.
  - `facies`: the facies column.
- Make the following plots:
  - Plot GR on its own. What does the _x_ axis represent?
  - Plot GR vs depth (ie with depth on the _x_ axis).
  - Plot depth vs GR.
  - Add `color='red'` to your call to `ax.plot()`.
  - What happens if you add another line with `ax.set_ylim(840, 920)`?
  - Switch the values in `ax.set_ylim()`.
  - Try instantiating the figure with `plt.subplots(figsize=(2,10))` at the start.
</div>

---
## Subplots

We can add another subplot:

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(4, 6), sharey=True)

ax0.plot(gr, depth, lw=0.5)
ax0.set_ylim(depth[-1]+10, depth[0]-10)
ax0.set_xlabel('GR [API]')
ax0.set_ylabel('Depth [m]')
ax0.set_title('GR log')
ax0.grid('k', alpha=0.3)

ax1.plot(phi, depth, 'g', lw=0.5)
ax1.set_ylim(depth[-1]+10, depth[0]-10)
ax1.set_xlabel('Phi [v/v]')
ax1.set_title('Phi log')
ax1.grid('k', alpha=0.3)

fig.show()  # <--- Note that you need this in a script.

## Save a figure

We often want to save figures as files. There are really only two sensible formats for saving scientific graphics:

- **PNG** — losslessly compressed rasters with defined resolution, which 'always work'.
- **SVG** — scalable vector graphics, which work at any scale but can display unpredictably.

In general, if you want to edit graphics before using them, export SVG and edit with Inkscape. Otherwise, make PNGs with a lot of pixels (at least 200 'dpi' whatever that means in 2025) and you can't go too far wrong. Don't use JPEG for graphics.

## `plt.scatter()`

It's also easy to make scatter plots:

We can adjust how the points plot to make it more interesting:

In [ ]:
fig, ax = plt.subplots()

ax.scatter(gr, phi, c=facies, s=10, alpha=0.5, cmap='tab10', label=facies)
ax.legend()
ax.grid(c='k', alpha=0.1)

## `plt.hist()`

We often want to look at the distribution of our data.

Note that we can also pass Pandas objects, eg Series, to `matplotlib`:

In general, `seaborn` makes nicer histograms, and adds a KDE (kernel density estimation) plot. It also prefers data without NaNs though...

In [ ]:
import seaborn as sns

ax = sns.histplot(gr, kde=True, lw=0)

We can also pass DataFrames to Seaborn, with a lot of extra options, eg using columns directly for colour etc.

In [ ]:
ax = sns.kdeplot(df, x='GR', hue='Well Name')

## `plt.imshow()` and contour maps

For image-like data, such as intensity maps or slices of seismic, we need a different kind of visualization. 

NB There's also `plt.pcolor` but it's very slow. Use `plt.pcolormesh` instead.

Let's load a seismic horizon. We have a DAT file, exported from OpendTect. We've left it in its original format, which looks like this:

```
# 1: X
# 2: Y
# 3: Inline
# 4: Crossline
# 5: Z
# - - - - - - - - - -
612076.10	6073980.89	110	550	1069.591283798217773
612126.08	6073982.28	110	552	1069.051265716552734
612176.06	6073983.68	110	554	1068.311929702758789
612226.04	6073985.08	110	556	1067.647337913513184
.
.
.
626557.62	6085589.72	558	1142	825.678706169128418
626607.60	6085591.12	558	1144	823.747575283050537
626657.58	6085592.52	558	1146	821.8720555305481
626707.56	6085593.91	558	1148	820.536375045776367
```

We made a small library called `gio` (very much a work in progress!) to read files like this. It produces an `xarray.Dataset`, which is a collection of NumPy-array-like things that have Pandas-like indexing wrapped around them. Sounds weird, but they are very useful!

In [ ]:
import requests
import gio
import pathlib

url = "https://github.com/scienxlab/datasets/raw/refs/heads/main/nlog/F3_Demo_0_FS4.dat"
fname = pathlib.Path('F3_Demo_0_FS4.dat')
with fname.open(mode='wt') as f:
    f.write(requests.get(url).text)
data = gio.read_odt(fname)['twt']
data

One nice thing about DataArrays is that they can plot themselves:

The colorbar orientation is a problem though: I'd prefer it to be the other way up.

Another nice thing is that they can give us their NumPy data; we'll get this and carry on in pure NumPy.

In [ ]:
hz = data.to_numpy()

hz

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">

<h3>EXERCISE</h3>

- Plot this dataset using `imshow()`
- Choose another colormap than the default.
- Add a colorbar to the side of the plot.
- Add a title.
- Stretch goal: add a second plot beneath the first using `fig, (ax0, ax1) = plt.subplots(nrows=2)` then plotting the horizon into `ax0` and a profile through the horizon as a line plot in `ax1`. Use `ax0.axhline()` to add a line to show the position of the profile.
</div>

In [ ]:
### YOUR CODE HERE



There's a 'filled contour' option too, `contourf`. Notice that it is plotted with the rows in a different order.

We can also use `plt.contour()`

It's often nice to combine these:

## Images

Images are a bit special: monochrome images are 2D of shape w &times; h, while colour images are 3D datasets of shape w &times; h &times; c, where c is some number of frequency 'channels', usually 3 (RGB) or 4 (RGBA).

Let's look at an RGB image (JPEGs are nearly always 3 channels).

In [ ]:
import urllib
from PIL import Image

url = 'https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/thinsectiongabbro.jpg'
img = Image.open(urllib.request.urlopen(url))

img

In [ ]:
import numpy as np

arr = np.asarray(img)

arr.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(arr[:, :, 0], cmap='gray')

In [ ]:
import seaborn as sns

sns.kdeplot(arr.reshape(-1, 3)[::100], palette='turbo_r')

## Interactive plots

The `ipywidgets` package gives us an easy way to make interactive plots. 

About the simplest thing you can do is like this:

In [ ]:
from ipywidgets import interact

@interact(a=(0, 10, 1), b=(0, 100, 10))
def main(a, b):
    """Do the things!"""
    print(a + b)
    return

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">

<h3>EXERCISE</h3>

Adapt the code above to make a slider for the profile plot of the horizon, eg choosing the profile number with the slider.

You will need to write a function that handles all of the plotting, from creating the figure and axis to plotting the objects.
</div>

## How complex do you want to get?

It turns out you can do almost anything in `matplotlib`. This is a single `matplotlib` figure:

In [ ]:
from IPython.display import Image
Image('https://raw.githubusercontent.com/agilescientific/geocomputing/refs/heads/develop/images/t1.png')

The key method you need to make a tiled plot like this is [`gridspec`](https://matplotlib.org/users/gridspec.html). You will also need a lot of patience.

## Getting help

- Gallery: https://matplotlib.org/stable/gallery/index.html
- Cheatsheets: https://matplotlib.org/cheatsheets/
- Book: https://github.com/rougier/scientific-visualization-book

---
Matt Hall, Equinor 2025 / Please share and re-use